<a href="https://colab.research.google.com/github/putriyunelfi/Machine-Learning/blob/main/FINAL%20EXAM/paper1_UAS_ML_MnistSimpleCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# imports 
import sys
import os
import argparse
import numpy as np 
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim #optimasi
import random #pemilihan acak
import torchvision.transforms.functional as A #import transform

from PIL import Image
from torchvision import transforms
from torchvision import datasets, transforms
from torchsummary import summary

In [16]:
#membuat class EMA untuk dipanggil pada saat train
class EMA:
    def __init__(self, model, decay): #inisialisasi 
        self.decay = decay
        self.shadow = {}
        self.original = {}

        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def __call__(self, model, num_updates): #memanggil update
        decay = min(self.decay, (1.0 + num_updates) / (10.0 + num_updates))
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - decay) * param.data + decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def assign(self, model): #menetapkan parameter
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.original[name] = param.data.clone()
                param.data = self.shadow[name]

    def resume(self, model): #lanjutan
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                param.data = self.original[name]

In [17]:
#Membuat kelas transform untuk dipanggil pada saat train data
class RandomRotation(object):
    def __init__(self, degrees, seed=1):
        self.degrees = (-degrees, degrees)
        random.seed(seed)
    
    @staticmethod #memanggil fungsi secara langsung
    def get_params(degrees):
        angle = random.uniform(degrees[0], degrees[1])
        return angle #mengembalikan nilai

    def __call__(self, img):
        angle = self.get_params(self.degrees)
        return A.rotate(img, angle, False, False, None, None) #mengembalika nilai

In [18]:
#membuat kelas dataset 
class MnistDataset(torch.utils.data.Dataset):
    def __init__(self, training=True, transform=None):
        if training==True:
          #membuka file data yang tersimpan pada drive
            f = open('/content/drive/MyDrive/MnistSimpleCNN-master/data/MNIST/raw/train-images-idx3-ubyte', 'rb')
            xs = np.array(np.frombuffer(f.read(), np.uint8, offset=16))
            f.close()
            f = open('/content/drive/MyDrive/MnistSimpleCNN-master/data/MNIST/raw/train-labels-idx1-ubyte', 'rb')
            ys = np.array(np.frombuffer(f.read(), np.uint8, offset=8))
            f.close()
        else:
            f = open('/content/drive/MyDrive/MnistSimpleCNN-master/data/MNIST/raw/t10k-images-idx3-ubyte', 'rb')
            xs = np.array(np.frombuffer(f.read(), np.uint8, offset=16))
            f.close()
            f = open('/content/drive/MyDrive/MnistSimpleCNN-master/data/MNIST/raw/t10k-labels-idx1-ubyte', 'rb')
            ys = np.array(np.frombuffer(f.read(), np.uint8, offset=8))
            f.close()
        xs = np.reshape(xs, (-1, 28, 28, 1)).astype(np.float32)
        ys = ys.astype(np.int)
        self.x_data = xs
        self.y_data = ys
        self.transform = transform #menyamakan variabel

    def __len__(self): #metode yang mendefinisikan objek
        return len(self.x_data)

    def __getitem__(self, idx): #indeks akses rentang nilai
        x = Image.fromarray(self.x_data[idx].reshape(28, 28))
        y = torch.tensor(np.array(self.y_data[idx]))
        if self.transform:
            x = self.transform(x)
        x = transforms.ToTensor()(np.array(x)/255)
        return x, y #akhir fungsi

In [19]:
#membuat kelas model M3, untuk pilihan logdir pada saat train data
class ModelM3(nn.Module):
    def __init__(self): #inisialisasi fungsi
        super(ModelM3, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, bias=False)       # output becomes 26x26
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 48, 3, bias=False)      # output becomes 24x24
        self.conv2_bn = nn.BatchNorm2d(48)
        self.conv3 = nn.Conv2d(48, 64, 3, bias=False)      # output becomes 22x22
        self.conv3_bn = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 80, 3, bias=False)      # output becomes 20x20
        self.conv4_bn = nn.BatchNorm2d(80)
        self.conv5 = nn.Conv2d(80, 96, 3, bias=False)      # output becomes 18x18
        self.conv5_bn = nn.BatchNorm2d(96)
        self.conv6 = nn.Conv2d(96, 112, 3, bias=False)     # output becomes 16x16
        self.conv6_bn = nn.BatchNorm2d(112)
        self.conv7 = nn.Conv2d(112, 128, 3, bias=False)    # output becomes 14x14
        self.conv7_bn = nn.BatchNorm2d(128)
        self.conv8 = nn.Conv2d(128, 144, 3, bias=False)    # output becomes 12x12
        self.conv8_bn = nn.BatchNorm2d(144)
        self.conv9 = nn.Conv2d(144, 160, 3, bias=False)    # output becomes 10x10
        self.conv9_bn = nn.BatchNorm2d(160)
        self.conv10 = nn.Conv2d(160, 176, 3, bias=False)   # output becomes 8x8
        self.conv10_bn = nn.BatchNorm2d(176)
        self.fc1 = nn.Linear(11264, 10, bias=False)
        self.fc1_bn = nn.BatchNorm1d(10)

    def get_logits(self, x): #mendapatlan log
        x = (x - 0.5) * 2.0
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        conv5 = F.relu(self.conv5_bn(self.conv5(conv4)))
        conv6 = F.relu(self.conv6_bn(self.conv6(conv5)))
        conv7 = F.relu(self.conv7_bn(self.conv7(conv6)))
        conv8 = F.relu(self.conv8_bn(self.conv8(conv7)))
        conv9 = F.relu(self.conv9_bn(self.conv9(conv8)))
        conv10 = F.relu(self.conv10_bn(self.conv10(conv9)))
        flat1 = torch.flatten(conv10.permute(0, 2, 3, 1), 1)
        logits = self.fc1_bn(self.fc1(flat1))
        return logits # akhir fungsi

    def forward(self, x): # mendapatkan outpun neural net
        logits = self.get_logits(x)
        return F.log_softmax(logits, dim=1)

In [20]:
#membuat kelas model M5, untuk pilihan logdir pada saat train data
class ModelM5(nn.Module):
    def __init__(self):
        super(ModelM5, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, bias=False)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 5, bias=False)
        self.conv2_bn = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 96, 5, bias=False)
        self.conv3_bn = nn.BatchNorm2d(96)
        self.conv4 = nn.Conv2d(96, 128, 5, bias=False)
        self.conv4_bn = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 160, 5, bias=False)
        self.conv5_bn = nn.BatchNorm2d(160)
        self.fc1 = nn.Linear(10240, 10, bias=False)
        self.fc1_bn = nn.BatchNorm1d(10)
        
    def get_logits(self, x):
        x = (x - 0.5) * 2.0
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        conv5 = F.relu(self.conv5_bn(self.conv5(conv4)))
        flat5 = torch.flatten(conv5.permute(0, 2, 3, 1), 1)
        logits = self.fc1_bn(self.fc1(flat5))
        return logits
        
    def forward(self, x):
        logits = self.get_logits(x)
        return F.log_softmax(logits, dim=1)

In [21]:
#membuat kelas model M7, untuk pilihan logdir pada saat train data

class ModelM7(nn.Module):
    def __init__(self):
        super(ModelM7, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, 7, bias=False)    # output becomes 22x22
        self.conv1_bn = nn.BatchNorm2d(48)
        self.conv2 = nn.Conv2d(48, 96, 7, bias=False)   # output becomes 16x16
        self.conv2_bn = nn.BatchNorm2d(96)
        self.conv3 = nn.Conv2d(96, 144, 7, bias=False)  # output becomes 10x10
        self.conv3_bn = nn.BatchNorm2d(144)
        self.conv4 = nn.Conv2d(144, 192, 7, bias=False) # output becomes 4x4
        self.conv4_bn = nn.BatchNorm2d(192)
        self.fc1 = nn.Linear(3072, 10, bias=False)
        self.fc1_bn = nn.BatchNorm1d(10)
    def get_logits(self, x):
        x = (x - 0.5) * 2.0
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        flat1 = torch.flatten(conv4.permute(0, 2, 3, 1), 1)
        logits = self.fc1_bn(self.fc1(flat1))
        return logits
    def forward(self, x):
        logits = self.get_logits(x)
        return F.log_softmax(logits, dim=1)

In [25]:
#menjalankan percobaan (train)

def run(p_seed=0, p_epochs=150, p_kernel_size=5, p_logdir="temp"):
    # nomor acak pada generator seed
    SEED = p_seed
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    np.random.seed(SEED)

    # kernel size model
    KERNEL_SIZE = p_kernel_size

    # nomor epochs
    NUM_EPOCHS = p_epochs

    # mengambil alamat nama file log pada drive
    if not os.path.exists("/content/drive/MyDrive/MnistSimpleCNN-master/logs/%s"%p_logdir):
        os.makedirs("/content/drive/MyDrive/MnistSimpleCNN-master/logs/%s"%p_logdir)
    OUTPUT_FILE = str("/content/drive/MyDrive/MnistSimpleCNN-master/logs/%s/log%03d.out"%(p_logdir,SEED))
    MODEL_FILE = str("/content/drive/MyDrive/MnistSimpleCNN-master/logs/%s/model%03d.pth"%(p_logdir,SEED))

    # mengaktifkan GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda == False:
        print("WARNING: CPU will be used for training.")
        exit(0)

    # metode augmentasi data
    transform = transforms.Compose([
        RandomRotation(20, seed=SEED),
        transforms.RandomAffine(0, translate=(0.2, 0.2)),
        ])

    # memuat data 
    train_dataset = MnistDataset(training=True, transform=transform)
    test_dataset = MnistDataset(training=False, transform=None)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=120, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

    # seleksi model
    if(KERNEL_SIZE == 3):
        model = ModelM3().to(device)
    elif(KERNEL_SIZE == 5):
        model = ModelM5().to(device)
    elif(KERNEL_SIZE == 7):
        model = ModelM7().to(device)

    summary(model, (1, 28, 28))

    # memilih hyperparameter
    ema = EMA(model, decay=0.999)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

    # menghapus file hasil
    f = open(OUTPUT_FILE, 'w')
    f.close()

    # variabel global
    g_step = 0
    max_correct = 0

    # pengulangan percobaan (training) dan evaluasi
    for epoch in range(NUM_EPOCHS):
        # proses training                                                         
        model.train()
        train_loss = 0
        train_corr = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            train_pred = output.argmax(dim=1, keepdim=True)
            train_corr += train_pred.eq(target.view_as(train_pred)).sum().item()
            train_loss += F.nll_loss(output, target, reduction='sum').item()
            loss.backward()
            optimizer.step()
            g_step += 1
            ema(model, g_step)
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{:05d}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
        train_loss /= len(train_loader.dataset)
        train_accuracy = 100 * train_corr / len(train_loader.dataset)

        # proses test                                                            
        model.eval()
        ema.assign(model)
        test_loss = 0
        correct = 0
        total_pred = np.zeros(0)
        total_target = np.zeros(0)
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += F.nll_loss(output, target, reduction='sum').item()
                pred = output.argmax(dim=1, keepdim=True)
                total_pred = np.append(total_pred, pred.cpu().numpy())
                total_target = np.append(total_target, target.cpu().numpy())
                correct += pred.eq(target.view_as(pred)).sum().item()
            if(max_correct < correct):
                torch.save(model.state_dict(), MODEL_FILE)
                max_correct = correct
                print("Best accuracy! correct images: %5d"%correct)
        ema.resume(model)

        # output                                                                   
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100 * correct / len(test_loader.dataset)
        best_test_accuracy = 100 * max_correct / len(test_loader.dataset)

        #menampilkan hasil akurasi tes
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%) (best: {:.2f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), test_accuracy, best_test_accuracy))

        f = open(OUTPUT_FILE, 'a')
        f.write(" %3d %12.6f %9.3f %12.6f %9.3f %9.3f\n"%(epoch, train_loss, train_accuracy, test_loss, test_accuracy, best_test_accuracy))
        f.close()

        # update penjadwal tingkat pembelajaran                                           
        lr_scheduler.step()

#seed untuk menentukan nilai awal
p_seed = int(input ("Seeds: ")) # memasukkan angka sebagai input seed

#epoch digunakan untuk melakukan pengulangan (sesuai input) terhadap trial yang dijalankan
p_epoch = int(input ("Epoch: ")) # memasukkan angka sebagai input epoch

#trial digunakan untuk pengulangan percobaan
p_trials = int(input ("Trials: ")) # memasukkan angka sebagai input epoch

p_kernel_size = int (input ("Kernel size: ")) #memasukkan angka sebagai input epoch (disesuaikan dengan angka pada model)

#gpu untuk menjalankan program pada grafik card
p_gpu = int(input ("GPU: ")) #masukkan angka sebagai input GPU

p_logdir = input ("Logdir: ") #masukkan input model yang akan digunakan

#menjalankan GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=str(p_gpu)

for x in range (p_trials): #pengulangan percobaan (trial)
  run(p_seed + x, p_epoch, p_kernel_size, p_logdir)

Seeds: 0
Epoch: 6
Trials: 4
Kernel size: 7
GPU: 0
Logdir: modelM7
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 48, 22, 22]           2,352
       BatchNorm2d-2           [-1, 48, 22, 22]              96
            Conv2d-3           [-1, 96, 16, 16]         225,792
       BatchNorm2d-4           [-1, 96, 16, 16]             192
            Conv2d-5          [-1, 144, 10, 10]         677,376
       BatchNorm2d-6          [-1, 144, 10, 10]             288
            Conv2d-7            [-1, 192, 4, 4]       1,354,752
       BatchNorm2d-8            [-1, 192, 4, 4]             384
            Linear-9                   [-1, 10]          30,720
      BatchNorm1d-10                   [-1, 10]              20
Total params: 2,291,972
Trainable params: 2,291,972
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Fo

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:992: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.594838
Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.520621
Train Epoch: 0 [36000/60000 (60%)]	Loss: 0.323656
Train Epoch: 0 [48000/60000 (80%)]	Loss: 0.372170
Best accuracy! correct images:  9903

Test set: Average loss: 0.1359, Accuracy: 9903/10000 (99.03%) (best: 99.03%)

Train Epoch: 1 [00000/60000 (0%)]	Loss: 0.249459
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.193397
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.254284
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.220086
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.199676
Best accuracy! correct images:  9924

Test set: Average loss: 0.0841, Accuracy: 9924/10000 (99.24%) (best: 99.24%)

Train Epoch: 2 [00000/60000 (0%)]	Loss: 0.138465
Train Epoch: 2 [12000/60000 (20%)]	Loss: 0.145364
Train Epoch: 2 [24000/60000 (40%)]	Loss: 0.161372
Train Epoch: 2 [36000/60000 (60%)]	Loss: 0.103480
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.251227

Test set: Average loss: 0.0937, Accuracy: 9891/10000 (98.91%) (best: 

In [26]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

#fungsi untuk run test data
def run(p_seed=0, p_kernel_size=5, p_logdir="temp"):

    # mengaktifkan GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda == False:
        print("WARNING: CPU will be used for training.")
        exit(0)

    # menjalankan data
    test_dataset = MnistDataset(training=False, transform=None)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

    # seleksi model yang digunakan
    if(p_kernel_size == 3):
        model1 = ModelM3().to(device)
    elif(p_kernel_size == 5):
        model1 = ModelM5().to(device)
    elif(p_kernel_size == 7):
        model1 = ModelM7().to(device)

    #mengambil data pada drive
    model1.load_state_dict(torch.load("/content/drive/MyDrive/MnistSimpleCNN-master/logs/%s/model%03d.pth"%(p_logdir,p_seed)))

    model1.eval()
    test_loss = 0
    correct = 0
    wrong_images = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model1(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            wrong_images.extend(np.nonzero(~pred.eq(target.view_as(pred)).cpu().numpy())[0]+(100*batch_idx))

    # menyimpan data pada drive
    np.savetxt("/content/drive/MyDrive/MnistSimpleCNN-master/logs/%s/wrong%03d.txt"%(p_logdir,p_seed), wrong_images, fmt="%d")
    # menampilkan gambar yang salah penebakan
    print(len(wrong_images), wrong_images)

p_logdir = input ("Logdir: ") #masukkan input model yang akan digunakan
p_seed = int(input ("Seeds: ")) # input angka untuk seed
p_trials = int(input ("Trials: ")) # masukkan angka sebagi input berapa kali percobaan
p_kernel_size = int (input ("Kernel size: ")) # masukkan angka sebagai input ukuran kernel (menyesuaikan angka pada model)

for y in range (p_trials): #perulangan percobaan (trial)
  run(p_seed + y, p_kernel_size, p_logdir)

Logdir: modelM7
Seeds: 0
Trials: 4
Kernel size: 7
64 [359, 445, 582, 625, 674, 716, 882, 1226, 1232, 1364, 1393, 1737, 1868, 1878, 1901, 2035, 2040, 2130, 2293, 2406, 2414, 2462, 2597, 2654, 2760, 2939, 3060, 3073, 3225, 3422, 3534, 3558, 3762, 3808, 3821, 3869, 3985, 4201, 4284, 4369, 4497, 4547, 4761, 4823, 5654, 5937, 6558, 6571, 6576, 6625, 8275, 8279, 8316, 8325, 8326, 8408, 9009, 9015, 9019, 9024, 9679, 9729, 9749, 9754]
78 [18, 359, 447, 449, 582, 619, 659, 674, 696, 716, 938, 1014, 1112, 1226, 1232, 1459, 1621, 1681, 1709, 1737, 1868, 1901, 2018, 2035, 2040, 2130, 2148, 2182, 2280, 2293, 2343, 2462, 2488, 2582, 2597, 2939, 3073, 3225, 3266, 3422, 3534, 3558, 3601, 3762, 3859, 4163, 4176, 4201, 4369, 4443, 4497, 4507, 4740, 4759, 4761, 4860, 5457, 5654, 5937, 5955, 6571, 6572, 6597, 6883, 7216, 8094, 8275, 8316, 8325, 8408, 8527, 9530, 9540, 9595, 9664, 9679, 9729, 9754]
71 [447, 582, 625, 674, 726, 846, 947, 1112, 1156, 1226, 1299, 1328, 1393, 1522, 1621, 1737, 1754, 1868, 1901

In [27]:
#inisialisai nilai variabel
cnt = 1
best = 10000
curr = 10000

p_kernel_size = int (input ("Kernel size: ")) #input ukuran kernel 

KERNEL_SIZE = p_kernel_size #menyamakan variabel

for i in range(4): #perulangan train (mengikuti jumlah trial yang digunakan)
    for j in range(i+1,4):
        for k in range(j+1,4):
          #mengambil data dari drive
            w1 = np.loadtxt("/content/drive/MyDrive/MnistSimpleCNN-master/logs/modelM%d/wrong%03d.txt"%(KERNEL_SIZE, i)).astype(np.int)
            w2 = np.loadtxt("/content/drive/MyDrive/MnistSimpleCNN-master/logs/modelM%d/wrong%03d.txt"%(KERNEL_SIZE, j)).astype(np.int)
            w3 = np.loadtxt("/content/drive/MyDrive/MnistSimpleCNN-master/logs/modelM%d/wrong%03d.txt"%(KERNEL_SIZE, k)).astype(np.int)

            board = np.zeros((10000))
            board[w1] += 1
            board[w2] += 1
            board[w3] += 1
            board = board // 2
            curr = np.sum(board)
            if curr < best:
                best = curr
            print("%4d %4d %4d %4d %4d %4d"%(cnt, len(w1), len(w2), len(w3), curr, best))
            cnt += 1

Kernel size: 7
   1   64   78   71   59   59
   2   64   78   95   67   59
   3   64   71   95   66   59
   4   78   71   95   70   59
